# Planetary boundaries: SSP indicators

#### Import packages

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#### Import data

In [2]:
index_rename_map =  {
    'Biodiversity': 'Biosphere<br>integrity',
    'Climate change': 'Climate<br>change',
    'Stratospheric ozone': 'Stratospheric<br>ozone',
    'Aerosols': 'Aerosols',
    'Ocean Acidification': 'Ocean<br>acidification',
    'Nitrogen balance': 'Nitrogen<br>balance',
    'Phosphorous balance': 'Phosphorous<br>balance',
    'Freshwater use': 'Freshwater<br>use',
    'Land system change': 'Land system<br>change',
}

In [36]:
def load_data(filename):
    pb_values_raw = pd.read_excel(filename, sheet_name="Values in Figures", header=0)
    pb_values_raw.columns = ['Name', 'Region', 'Scenario'] + pb_values_raw.columns[3:].tolist()
    pb_values_raw = pb_values_raw.replace(index_rename_map).drop(columns='Region').set_index(['Scenario', 'Name'])
    return pb_values_raw

pb_values_raw = load_data('Data/summary - 20240529.xlsx')
# pb_values_raw = load_data('Data/summary - 20240703.xlsx')
pb_values_raw.head()

1970      2015      2030      2050  \
Scenario Name                                                             
SSP1     Climate<br>change       0.514000  1.825273  2.455273  2.962364   
         Stratospheric<br>ozone  0.157970  0.902687  0.599751  0.326132   
         Aerosols                2.012666  2.061741  1.927661  1.694579   
         Ocean<br>acidification  0.455601  0.813883  0.989778  1.477498   
         Nitrogen<br>balance     1.337535  2.602917  2.603702  2.452318   

                                     2100  
Scenario Name                              
SSP1     Climate<br>change       4.014636  
         Stratospheric<br>ozone  0.157970  
         Aerosols                1.259313  
         Ocean<br>acidification  2.405665  
         Nitrogen<br>balance     2.414548

In [13]:
# pb_values_raw_old = load_data('Data/summary - 20240529.xlsx')

In [38]:
def get_pb_values(SSP):
    return pb_values_raw.loc[SSP].loc[index_rename_map.values()]

#### General Planetary Boundaries-plot

In [39]:
LEGEND_LABELS = [
    "(A) Below boundary (safe)",
    "(B) In zone of uncertainty (increasing risk)",
    "(C) Beyond zone of uncertainty (high risk)",
]

In [40]:
def add_pb_subplot(data, column, fig, subplot='polar', subplot2='polar2', domain_x=[0,1], domain_y=[0,1], max_val=3.6, bg_limit = 3, show_safety_grid=True):
    
    n = len(data)
    beginthetas = np.arange(n) * 360 / n
    midthetas = beginthetas + 360/(2*n)
    category_names = data.index.values
    
    background_r = bg_limit

    ## Background
    fig.add_trace(go.Barpolar(
        r=[background_r], theta=[0], width=[360], marker_color='#E9F0FA',
        showlegend=False, subplot=subplot
    ))

    ## Data
    for maxval, minval, color, name in zip(
        [1,1,100], [0, 1, 2],
        ['#86A851', '#FAB800', '#F07204'],
        LEGEND_LABELS,
    ):
        rvalues = (data-minval).clip(0, maxval)[column].values
        fig.add_trace(go.Barpolar(
            r=rvalues, theta=midthetas,
            width=360/n, marker_color=color,
            subplot=subplot2, marker_line_width=0, name=name, showlegend=subplot=='polar'
        ))


    _num = 100
    ## Circular grid lines
    # Safety zone, r=1
    if show_safety_grid:
        fig.add_trace(go.Scatterpolar(
            r=np.ones(_num)*1, theta=np.linspace(0,360,_num), showlegend=False,
            line={'color': 'black', 'dash': 'dot', 'width': 2}, subplot=subplot2
        ))
        # Uncertainty zone, r=2
        fig.add_trace(go.Scatterpolar(
            r=np.ones(_num)*2, theta=np.linspace(0,360,_num), showlegend=False,
            line={'color': '#B1AFB2', 'dash': 'dot', 'width': 2}, subplot=subplot2
        ))
    # Other lines
    grid_rs = [1.5] + list(np.arange(2.5, max_val-0.5, 0.5))
    for r in grid_rs:
        fig.add_trace(go.Scatterpolar(
            r=np.ones(_num)*r, theta=np.linspace(0,360,_num), showlegend=False,
            line_color='#E3E4E6', line_width=0.7, subplot=subplot2
        ))

    ## Category separators
    for i in range(n):
        theta = i * 360 / n
        dtheta = 0.7
        fig.add_trace(go.Scatterpolar(
            r=[max_val, 0., max_val], theta=[theta - dtheta, theta, theta + dtheta],
            fill='toself', mode='lines', fillcolor='black', line_width=0,
            showlegend=False, subplot=subplot2
        ))
        
    ## Return the layout values for this subplot
    return {
        subplot2: {
            'radialaxis_range': [0,max_val], 'hole': 0, 'radialaxis_visible': False,
            'angularaxis': {
                'rotation': 180, 'direction': 'clockwise',
                'showgrid': False, 'tickvals': midthetas, 'ticktext': category_names, 'ticks': 'inside', 'ticklen': 10
            },
            'bgcolor': 'rgba(0,0,0,0)',
            'domain': {'x': domain_x, 'y': domain_y}
        },
        subplot: {
            'domain': {'x': domain_x, 'y': domain_y},
            'radialaxis_range': [0,max_val], 'hole': 0, 'radialaxis_visible': False,
            'angularaxis_visible': False, 'bgcolor': 'rgba(0,0,0,0)'
        }
    }


## SSP2 over the years

In [41]:
data = get_pb_values('SSP2')
data

,1970,2015,2030,2050,2100
Name,,,,,
Biosphere<br>integrity,2.236330,2.642420,2.799930,3.028310,3.615960
Climate<br>change,0.514019,1.825215,2.495222,3.168799,5.150088
Stratospheric<br>ozone,0.157970,0.902687,0.599751,0.326132,0.157970
Aerosols,2.012666,2.061713,2.019971,1.972460,1.385293
Ocean<br>acidification,0.455323,0.813726,1.002747,1.649333,3.317681
Nitrogen<br>balance,1.337534,2.602915,2.916341,3.193354,3.386890
Phosphorous<br>balance,1.270859,2.067643,2.437663,2.447953,2.561381
Freshwater<br>use,0.573639,0.859729,0.941425,1.025775,1.025469
Land system<br>change,1.369600,1.539171,1.631971,1.727686,1.886345


In [42]:
def range_xy(i, n, padding=0.03):
    width = (1 - padding * (n-1)) / n
    return [(i-1) * (width + padding), (i-1) * (width + padding) + width]

range_x = range_xy
# y-range is flipped (0 is below, 1 is up)
def range_y(i, n, padding=0.15):
    return range_xy(n-i+1, n, padding)

In [43]:
def left_align_subplot_titles(fig):
    """Should be called directly after calling `make_subplots(...)`,
    otherwise new annotations besides the subtitles can be created."""
    for i, ann in enumerate(fig.layout.annotations):
        x = fig.layout[f"xaxis{i+1}"].domain[0]
        ann.update(x=x, xanchor="left", align="left")


In [44]:
years = [1970, 2015, 2030, 2050]

n_x, n_y = 2, 2

fig = make_subplots(n_y, n_x, subplot_titles=['<b>SSP2 {}:</b>'.format(y) for y in years], vertical_spacing=0.1, horizontal_spacing=0.0)
left_align_subplot_titles(fig)

max_val = 3.6
bg_limit = 3

extra_layout1 = add_pb_subplot(data, years[0], fig, 'polar', 'polar2', range_x(1, n_x), range_y(1, n_y), max_val=max_val, bg_limit=bg_limit)
extra_layout2 = add_pb_subplot(data, years[1], fig, 'polar3', 'polar4', range_x(2, n_x), range_y(1, n_y), max_val=max_val, bg_limit=bg_limit)
extra_layout3 = add_pb_subplot(data, years[2], fig, 'polar5', 'polar6', range_x(1, n_x), range_y(2, n_y), max_val=max_val, bg_limit=bg_limit)
extra_layout4 = add_pb_subplot(data, years[3], fig, 'polar7', 'polar8', range_x(2, n_x), range_y(2, n_y), max_val=max_val, bg_limit=bg_limit)

fig.update_layout(
    **extra_layout1,
    **extra_layout2,
    **extra_layout3,
    **extra_layout4,
    height=700, width=850,
    margin={'b': 20, 'l': 50, 't': 80, 'r': 20},
    legend={'orientation': 'h', 'x': 0.5, "xanchor": "center"}
)


In [23]:
fig.write_image('figures/figure_1.svg', scale=2)

## Different SSPs

In [24]:
colors_PBL = ['#00AEEF', '#808D1D', '#B6036C', '#FAAD1E', '#3F1464', '#7CCFF2', '#F198C1', '#42B649', '#EE2A23', '#004019', '#F47321', '#511607', '#BA8912', '#78CBBF', '#FFF229', '#0071BB']

In [25]:
def add_pb_subplot_lines(data, reference, fig, subplot='polar', subplot2='polar2', domain_x=[0,1], domain_y=[0,1], max_val=3.6, bg_limit = 3, ticks = 'inside', colors = colors_PBL, dashes=None, ref_color='#AAA', show_safety_grid=True, showbg=True):
    
    colormap = dict(zip(data.columns, colors))
    if dashes is None:
        dashmap = dict(zip(data.columns, ['solid']*len(data.columns)))
    else:
        dashmap = dict(zip(data.columns, dashes))
    n = len(data)
    beginthetas = np.arange(n) * 360 / n
    midthetas = beginthetas + 360/(2*n)
    category_names = data.index.values
    
    background_r = bg_limit

    ## Background
    if showbg:
        fig.add_trace(go.Barpolar(
            r=[background_r], theta=[0], width=[360], marker_color='#E9F0FA',
            showlegend=False, subplot=subplot
        ))
    
    _num = 100
    
    
    
    ## Circular grid lines
    # Safety zone, r=1
    if show_safety_grid:
        fig.add_trace(go.Scatterpolar(
            r=np.ones(_num)*1, theta=np.linspace(0,360,_num), showlegend=False,
            line={'color': 'black', 'dash': 'dot', 'width': 3}, subplot=subplot2
        ))
        # Uncertainty zone, r=2
        fig.add_trace(go.Scatterpolar(
            r=np.ones(_num)*2, theta=np.linspace(0,360,_num), showlegend=False,
            line={'color': '#B1AFB2', 'dash': 'dot', 'width': 3}, subplot=subplot2
        ))
    
    ## Reference
    if reference is not None:
        fig.add_barpolar(
            r=reference.values, theta=midthetas,
            width=360/n, marker_color=ref_color,
            subplot=subplot2, marker_line_width=0, name=reference.name, showlegend=subplot=='polar'
        )

    ## Data
    for i, midtheta in enumerate(midthetas):
        
        rounded = (data.iloc[i] * 9).round(decimals=0) / 9
        
        j = 0
        
        for r_round in rounded.unique():
            columns = rounded[rounded == r_round].index.values
            num_overlaps = len(columns)
            for k, (name, column) in enumerate(data[columns].items()):
                color = colormap[name]
                dash = dashmap[name]
                j += 1
                
                r = column.values[i]
                # if r > max_val:
                #     r = max_val - 0.01
                #     line_style = 'dot'
                # else:
                #     line_style = 'solid'
                line_style = dash

                begin_theta_full, end_theta_full = midtheta - 360/(2*n), midtheta + 360/(2*n)
                
                delta_theta = (end_theta_full - begin_theta_full) / num_overlaps
                begin_theta = begin_theta_full + k * delta_theta
                end_theta = begin_theta + delta_theta

                fig.add_scatterpolar(
                    r = np.ones(_num)*r, theta=np.linspace(begin_theta, end_theta, _num),
                    line={'color': color, 'width': 4, 'dash': line_style},
                    showlegend=False, name=name, subplot=subplot2
                )

    if subplot == 'polar':
        for name, color in colormap.items():
            dash = dashmap[name]
            fig.add_scatterpolar(
                r=[None], theta=[None], line={'color': color, 'width': 4, 'dash': dash}, name=name, subplot=subplot2, mode='lines',
            )
    
    
    
    # Other lines
    grid_rs = [1.5] + list(np.arange(2.5, max_val-0.5, 0.5))
    for r in grid_rs:
        fig.add_trace(go.Scatterpolar(
            r=np.ones(_num)*r, theta=np.linspace(0,360,_num), showlegend=False,
            line_color='#E3E4E6', line_width=0.7, subplot=subplot
        ))

    ## Category separators
    for i in range(n):
        theta = i * 360 / n
        dtheta = 0.7
        fig.add_trace(go.Scatterpolar(
            r=[max_val, 0., max_val], theta=[theta - dtheta, theta, theta + dtheta],
            fill='toself', mode='lines', fillcolor='black', line_width=0,
            showlegend=False, subplot=subplot2
        ))
        
    ## Return the layout values for this subplot
    return {
        subplot2: {
            'radialaxis_range': [0,max_val], 'hole': 0, 'radialaxis_visible': False,
            'angularaxis': {
                'rotation': 180, 'direction': 'clockwise',
                'showgrid': False, 'tickvals': midthetas, 'ticktext': category_names, 'ticks': ticks, 'ticklen': 10
            },
            'bgcolor': 'rgba(0,0,0,0)',
            'domain': {'x': domain_x, 'y': domain_y}
        },
        subplot: {
            'domain': {'x': domain_x, 'y': domain_y},
            'radialaxis_range': [0,max_val], 'hole': 0, 'radialaxis_visible': False,
            'angularaxis_visible': False, 'bgcolor': 'rgba(0,0,0,0)'
        }
    }



In [26]:
data_SSPs = pd.DataFrame({
    '2015': get_pb_values('SSP2')[2015],
    'SSP1': get_pb_values('SSP1')[2050],
    'SSP2': get_pb_values('SSP2')[2050],
    'SSP3': get_pb_values('SSP3')[2050]
})
data_SSPs

,2015,SSP1,SSP2,SSP3
Name,,,,
Biosphere<br>integrity,2.196283,2.270606,2.393406,2.443337
Climate<br>change,1.825215,2.962364,3.168799,3.453367
Stratospheric<br>ozone,0.902687,0.326132,0.326132,0.240716
Aerosols,1.833333,1.406667,1.693333,1.813333
Ocean<br>acidification,0.813726,1.477498,1.649333,1.737577
Nitrogen<br>balance,2.602915,2.452318,3.193354,3.619577
Phosphorous<br>balance,2.067643,1.758580,2.447953,2.841289
Freshwater<br>use,0.859729,0.886440,1.025775,1.083969
Land system<br>change,1.539171,1.512228,1.727686,1.842855


In [28]:
years = [1970, 2015, 2030, 2050]

n_x, n_y = 1, 1

fig2 = make_subplots(n_y, n_x)
# left_align_subplot_titles(fig)

max_val = 3.7
bg_limit = 3

fig2.add_scatterpolar(r=[None], theta=[None], mode='lines', line={'color': 'rgba(0,0,0,0)'}, name='SSP2:')
extra_layout1 = add_pb_subplot(data_SSPs, "SSP2", fig2, 'polar', 'polar2', range_x(1, n_x), range_y(1, n_y), max_val=max_val, bg_limit=bg_limit, show_safety_grid=False)

# _ref = data_SSPs['2015']
fig2.add_scatterpolar(r=[None], theta=[None], mode='lines', line={'color': 'rgba(0,0,0,0)'}, name=' ')
_ref = None
extra_layout2 = add_pb_subplot_lines(
    data_SSPs.loc[:,['SSP1', 'SSP3', '2015']], _ref, fig2, 'polar', 'polar2', range_x(1, n_x), range_y(1, n_y),
    max_val=max_val, bg_limit=bg_limit, ticks=None, colors=['#333', '#AAA', '#666'], dashes=['solid', 'solid', '2px 2px'], ref_color='#CCC', show_safety_grid=False, showbg=False
)

fig2.update_layout(
    **extra_layout1,
    height=600, width=850,
    margin={'b': 20, 'l': 50, 't': 80, 'r': 20},
    legend={'orientation': 'h', 'x': 0.5, "xanchor": "center"}
)

In [29]:
fig2.write_image('figures/figure_2.svg', scale=2)

In [30]:
data_mitig = pd.DataFrame({
    '2015': get_pb_values('SSP2')[2015],
    'SSP2 (2030)': get_pb_values('SSP2')[2030],
    'SSP2 (2050)': get_pb_values('SSP2')[2050],
    'Mitigation (2030)': get_pb_values('SSP2_19')[2030],
    'Mitigation (2050)': get_pb_values('SSP2_19')[2050],
    'Sustainability (2030)': get_pb_values('SSP2_19_Sus')[2030],
    'Sustainability (2050)': get_pb_values('SSP2_19_Sus')[2050],

    'Sustainability* (2030)': get_pb_values('SSP2_Su')[2030],
    'Sustainability* (2050)': get_pb_values('SSP2_Su')[2050],
})
data_mitig

,2015,SSP2 (2030),SSP2 (2050),Mitigation (2030),Mitigation (2050),Sustainability (2030),Sustainability (2050),Sustainability* (2030),Sustainability* (2050)
Name,,,,,,,,,
Biosphere<br>integrity,2.196283,2.274551,2.393406,2.257300,2.276720,2.237766,2.199667,2.228274,2.231333
Climate<br>change,1.825215,2.495222,3.168799,2.432132,2.083401,2.439572,2.095362,2.464966,2.949215
Stratospheric<br>ozone,0.902687,0.599751,0.326132,0.697248,0.505505,0.697248,0.505505,0.599751,0.326132
Aerosols,1.833333,1.766667,1.693333,1.426667,1.166667,1.420000,1.020000,1.640000,1.493333
Ocean<br>acidification,0.813726,1.002747,1.649333,0.946328,1.002054,0.956707,1.072535,0.997149,1.594085
Nitrogen<br>balance,2.602915,2.916341,3.193354,2.671683,2.603902,2.422780,2.099100,2.477171,2.185236
Phosphorous<br>balance,2.067643,2.437663,2.447953,2.449534,2.325450,1.622427,0.729680,1.536643,0.880373
Freshwater<br>use,0.859729,0.941425,1.025775,0.864150,0.930439,0.742336,0.716748,0.794088,0.805010
Land system<br>change,1.539171,1.631971,1.727686,1.583056,1.404964,1.406520,1.105999,1.554976,1.405935


In [31]:
def make_fig3(with_nexus=False):

    n_x, n_y = 2, 1

    fig3 = make_subplots(n_y, n_x, subplot_titles=("<b>2030</b>", "<b>2050</b>"))

    max_val = 4.1
    bg_limit = 3

    padding = 0.15

    # colors = [colors_PBL[i] for i in [6,2,5,0]]
    # colors = [colors_PBL[i] for i in [2, 0, 1]]
    colors = [colors_PBL[2], colors_PBL[0], '#666', '#00FF00']
    dashes = ['solid', 'solid', '2px 2px', 'solid']

    extra_layout1 = add_pb_subplot(data_mitig, "SSP2 (2030)", fig3, 'polar', 'polar2', range_x(1, n_x, padding=padding), range_y(1, n_y, padding=padding), max_val=max_val, bg_limit=bg_limit, show_safety_grid=False)

    columns_2030 = ["Mitigation (2030)", "Sustainability (2030)", "2015"]
    if with_nexus:
        columns_2030 += ["Sustainability* (2030)"]

    extra_layout1 = add_pb_subplot_lines(
        data_mitig.loc[
            :, columns_2030
        ].rename(
            columns={
                "Mitigation (2030)": "Climate mitigation",
                "Sustainability (2030)": "Sustainability",
            }
        ),
        # data_mitig.loc[:,'SSP2 (2030)':],
        None,
        fig3,
        "polar",
        "polar2",
        range_x(1, n_x, padding=padding),
        range_y(1, n_y, padding=padding),
        max_val=max_val,
        bg_limit=bg_limit,
        ticks=None,
        colors=colors,
        dashes=dashes,
        ref_color="#CCC",
        showbg=False, show_safety_grid=False,
    )

    add_pb_subplot(data_mitig, "SSP2 (2050)", fig3, 'polar3', 'polar4', range_x(2, n_x, padding=padding), range_y(1, n_y, padding=padding), max_val=max_val, bg_limit=bg_limit, show_safety_grid=False)
    columns_2050 = ["Mitigation (2050)", "Sustainability (2050)", "2015"]
    if with_nexus:
        columns_2050 += ["Sustainability* (2050)"]

    extra_layout2 = add_pb_subplot_lines(
        data_mitig.loc[:, columns_2050],
        None,
        fig3,
        "polar3",
        "polar4",
        range_x(2, n_x, padding=padding),
        range_y(1, n_y, padding=padding),
        max_val=max_val,
        bg_limit=bg_limit,
        ticks=None,
        colors=colors,
        dashes=dashes,
        showbg=False,
        ref_color="#CCC", show_safety_grid=False,
    )


    fig3.update_layout(
        **extra_layout1,
        **extra_layout2,
        height=580,
        width=950,
        margin={"b": 150, "l": 80, "t": 40, "r": 50},
        # legend={'orientation': 'v', 'y': 0.5}
        legend={"orientation": "h", "x": 0.5, "xanchor": "center", "font_size": 13},
    )
    return fig3

fig3 = make_fig3()
fig3

In [32]:
fig3.write_image('figures/figure_3.svg', scale=2)

In [33]:
fig3_nexus = make_fig3(with_nexus=True)
fig3_nexus

In [34]:
fig3_nexus.write_image('figures/figure_3_nexus.svg', scale=2)
fig3_nexus.write_image('figures/figure_3_nexus.png', scale=2)